<a href="https://colab.research.google.com/github/BarbJS/Gerenciamento-de-biblioteca-digital/blob/main/gestao_arquivos_digitais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## *Projeto: Sistema de gerenciamento de biblioteca digital*

In [ ]:
# -*- coding: utf-8 -*-
# Importando as bibliotecas necessárias
import os  # Biblioteca para interagir com o sistema operacional (criar pastas, listar arquivos)
import shutil  # Biblioteca para operações de alto nível com arquivos (mover arquivos)
import re  # Biblioteca para trabalhar com expressões regulares (usaremos para achar o ano no nome do arquivo)
import datetime  # Biblioteca para trabalhar com datas e horas (usaremos para pegar o ano atual)

In [ ]:
# --- CONFIGURAÇÃO INICIAL DAS PASTAS ---
# Prints para guiar o usuário na configuração inicial
print("="*50)
print("Bem-vindo ao Sistema de Gestão de Documentos da Biblioteca!")
print("="*50)
print("\nPASSO 1: Vamos configurar as pastas de trabalho.")

# Define o nome da pasta onde os arquivos a serem organizados ficarão.
# Esta pasta será criada no ambiente do Colab.
PASTA_ORIGEM_LOCAL = 'arquivos_para_organizar'

# Define o nome da pasta onde os arquivos já organizados serão guardados.
PASTA_DESTINO = 'biblioteca_organizada'

# Verifica se a pasta de origem já existe. Se não existir, ela é criada.
if not os.path.exists(PASTA_ORIGEM_LOCAL):
    os.makedirs(PASTA_ORIGEM_LOCAL) # Cria a pasta.
    print(f"Pasta '{PASTA_ORIGEM_LOCAL}' criada com sucesso! Por favor, adicione seus arquivos aqui.")
else:
    print(f"Pasta '{PASTA_ORIGEM_LOCAL}' já existe. Pronto para receber os arquivos.")

# Verifica se a pasta de destino já existe. Se não existir, ela é criada.
if not os.path.exists(PASTA_DESTINO):
    os.makedirs(PASTA_DESTINO) # Cria a pasta.
    print(f"Pasta '{PASTA_DESTINO}' criada com sucesso! Aqui ficarão seus arquivos organizados.")
else:
    print(f"Pasta '{PASTA_DESTINO}' já existe. Os arquivos organizados serão salvos aqui.")

print("\nConfiguração inicial concluída!")
print("-" * 50)

In [ ]:
# --- FUNÇÕES AUXILIARES ---

def extrair_ano_do_arquivo(nome_arquivo):
    """
    Função para extrair o ano do nome de um arquivo.
    Procura por um número de 4 dígitos (ex: 2021, 2023).
    Se não encontrar, retorna o ano atual.
    """

    # Usamos uma expressão regular para encontrar um número de 4 dígitos que comece com 19 ou 20.
    # Isso ajuda a garantir que estamos pegando um ano válido.
    match = re.search(r'(19|20)\d{2}', nome_arquivo)

    # Se um ano for encontrado no nome do arquivo...
    if match:
        return match.group(0) # Retorna o ano encontrado (ex: "2021").
    # Se não for encontrado...
    else:
        # Pega a data e hora de hoje e extrai apenas o ano.
        ano_atual = datetime.datetime.now().year
        return str(ano_atual) # Retorna o ano atual como texto (ex: "2025").

def organizar_arquivos(pasta_origem, pasta_destino):
    """
    Função principal que organiza os arquivos.
    Move os arquivos da pasta de origem para a pasta de destino,
    criando subpastas por tipo de arquivo e ano de publicação.
    """
    print("\nIniciando a organização dos arquivos...")
    # Lista todos os arquivos e pastas dentro da pasta de origem.
    arquivos_na_origem = os.listdir(pasta_origem)

    # Verifica se há arquivos para organizar.
    if len(arquivos_na_origem) == 0:
        print("AVISO: A pasta de origem está vazia. Não há arquivos para organizar.")
        return # Encerra a função se não houver arquivos.

    # Laço de repetição 'for' para processar cada item na lista de arquivos.
    for nome_arquivo in arquivos_na_origem:
        # Monta o caminho completo para o arquivo de origem.
        caminho_completo_origem = os.path.join(pasta_origem, nome_arquivo)

        # Verifica se o item é realmente um arquivo (e não uma pasta).
        if os.path.isfile(caminho_completo_origem):
            # 1. Separar a extensão do arquivo (pdf, epub, etc.)
            # O ponto é removido e o texto é convertido para minúsculo.
            extensao = os.path.splitext(nome_arquivo)[1].lower().replace('.', '')
            if extensao == '': # Se for um arquivo sem extensão
                extensao = 'sem_extensao'

            # 2. Extrair o ano de publicação do nome do arquivo.
            ano = extrair_ano_do_arquivo(nome_arquivo)

            # 3. Criar as pastas de destino, se não existirem.
            # Primeiro, a pasta do tipo de arquivo (ex: .../biblioteca_organizada/pdf/)
            pasta_por_tipo = os.path.join(pasta_destino, extensao)
            if not os.path.exists(pasta_por_tipo):
                os.makedirs(pasta_por_tipo)

            # Depois, a pasta do ano (ex: .../biblioteca_organizada/pdf/2021/)
            pasta_por_ano = os.path.join(pasta_por_tipo, ano)
            if not os.path.exists(pasta_por_ano):
                os.makedirs(pasta_por_ano)

            # 4. Mover o arquivo para o destino final.
            caminho_completo_destino = os.path.join(pasta_por_ano, nome_arquivo)
            shutil.move(caminho_completo_origem, caminho_completo_destino)

            # Imprime uma mensagem de sucesso para o usuário.
            print(f"SUCESSO: Arquivo '{nome_arquivo}' movido para '{pasta_por_ano}'.")

    print("\nOrganização concluída com sucesso!")


def adicionar_novos_arquivos(pasta_destino_upload):
    """
    Função para permitir que o usuário adicione novos arquivos do seu computador.
    """
    print("\nAbra a janela de seleção de arquivos e escolha os documentos que deseja adicionar.")
    print("Aguarde o upload ser 100% concluído antes de continuar.")
    # Abre a interface de upload de arquivos do Google Colab.
    uploaded = files.upload()

    # Verifica se algum arquivo foi realmente enviado.
    if len(uploaded) == 0:
        print("Nenhum arquivo foi selecionado para upload.")
        return # Encerra a função.

    # Itera sobre os arquivos que foram enviados.
    for nome_arquivo in uploaded.keys():
        # Move o arquivo da pasta temporária do Colab para a nossa pasta de origem.
        shutil.move(nome_arquivo, os.path.join(pasta_destino_upload, nome_arquivo))
        print(f"SUCESSO: Arquivo '{nome_arquivo}' adicionado à pasta de organização.")


def buscar_e_renomear_arquivo(pasta_raiz, nome_antigo, nome_novo):
    """
    Busca um arquivo em toda a estrutura de pastas e o renomeia.
    """
    # A variável 'arquivo_encontrado' nos ajuda a saber se encontramos o arquivo.
    arquivo_encontrado = False

    # os.walk é uma ferramenta poderosa que "caminha" por todas as pastas e subpastas.
    for diretorio_atual, subpastas, arquivos in os.walk(pasta_raiz):
        # Verifica se o arquivo que queremos renomear está na lista de arquivos do diretório atual.
        if nome_antigo in arquivos:
            # Monta o caminho completo do arquivo antigo e do novo.
            caminho_antigo = os.path.join(diretorio_atual, nome_antigo)
            caminho_novo = os.path.join(diretorio_atual, nome_novo)

            # Renomeia o arquivo.
            os.rename(caminho_antigo, caminho_novo)
            print(f"SUCESSO: Arquivo '{nome_antigo}' renomeado para '{nome_novo}'.")
            arquivo_encontrado = True # Marcamos que encontramos e renomeamos o arquivo.
            break # Interrompe o loop 'for' pois já encontramos o arquivo.

    # Se o loop terminar e não tivermos encontrado o arquivo, informamos o usuário.
    if not arquivo_encontrado:
        print(f"ERRO: O arquivo '{nome_antigo}' não foi encontrado na biblioteca organizada.")


def buscar_e_remover_arquivo(pasta_raiz, nome_arquivo):
    """
    Busca um arquivo em toda a estrutura de pastas e o remove.
    """
    arquivo_encontrado = False

    # Novamente, usamos o os.walk para procurar em todas as pastas.
    for diretorio_atual, subpastas, arquivos in os.walk(pasta_raiz):
        # Verifica se o arquivo está no diretório atual.
        if nome_arquivo in arquivos:
            caminho_completo = os.path.join(diretorio_atual, nome_arquivo)
            # Remove o arquivo.
            os.remove(caminho_completo)
            print(f"SUCESSO: Arquivo '{nome_arquivo}' removido da biblioteca.")
            arquivo_encontrado = True # Marcamos que encontramos e removemos.
            break # Interrompe o loop.

    # Se o arquivo não for encontrado, informa o usuário.
    if not arquivo_encontrado:
        print(f"ERRO: O arquivo '{nome_arquivo}' não foi encontrado na biblioteca organizada.")

In [ ]:
#Função opcional a depender da preferência do usuário por usar o Google Drive

from google.colab import drive # Módulo específico do Google Colab para conectar com o Google Drive
from google.colab import files # Módulo específico do Google Colab para fazer upload de arquivos do computador

def conectar_google_drive():
    """
    Função para conectar o Colab ao Google Drive do usuário.
    """
    print("\nConectando ao Google Drive...")
    print("Uma janela de autorização do Google aparecerá. Por favor, siga as instruções.")
    drive.mount('/content/drive')
    print("SUCESSO: Google Drive conectado!")
    # Pede ao usuário o caminho da pasta no Drive
    caminho_drive = input("Por favor, cole aqui o caminho da pasta no seu Drive que contém os arquivos (ex: /content/drive/MyDrive/Documentos): ")
    return caminho_drive

In [ ]:
# --- INTERFACE DE COMANDO PRINCIPAL ---

def menu_principal():
    """
    Função que exibe o menu e controla o fluxo do programa.
    """
    # Define a pasta de origem padrão como a pasta local do Colab.
    pasta_origem_atual = PASTA_ORIGEM_LOCAL

    # O laço 'while True' cria um menu que fica sempre rodando até o usuário decidir sair.
    while True:
        print("\n" + "="*20 + " MENU PRINCIPAL DE COMANDOS " + "="*20)
        print("\n" + "Seja bem-vindo(a) ao Sistema de Gerenciamento da sua Biblioteca Digital!")
        print("Por favor, escolha uma opção:" + "\n")
        print("  1. Usar uma pasta do Google Drive como origem")
        print("  2. Organizar arquivos da pasta de origem")
        print("  3. Adicionar novos arquivos da pasta de origem")
        print("  4. Renomear um arquivo da pasta de destino")
        print("  5. Remover um arquivo da pasta de destino")
        print("  6. Sair do programa")
        print(f"\n(Pasta de origem atual: '{pasta_origem_atual}')")

        # Pede para o usuário digitar uma opção.
        escolha = input("Digite o número da sua escolha: ")

        # --- Opção 1: Google Drive ---
        if escolha == '1':
            caminho_gdrive = conectar_google_drive()
            # Verifica se o caminho informado pelo usuário realmente existe.
            if os.path.exists(caminho_gdrive):
                pasta_origem_atual = caminho_gdrive # Atualiza a pasta de origem para a do Drive.
                print(f"SUCESSO: Pasta de origem atualizada para '{pasta_origem_atual}'.")
                print("Agora você pode usar a opção 2 para organizar os arquivos desta pasta.")
            else:
                print(f"ERRO: O caminho '{caminho_gdrive}' não existe no seu Google Drive. Tente novamente.")

        # --- Opção 2: Organizar ---
        elif escolha == '2':
            # Antes de organizar, verifica se a pasta de origem existe.
            if not os.path.exists(pasta_origem_atual):
                print(f"ERRO: A pasta de origem '{pasta_origem_atual}' não foi encontrada. Verifique o caminho ou use a opção 1.")
            else:
                organizar_arquivos(pasta_origem_atual, PASTA_DESTINO)

        # --- Opção 3: Adicionar ---
        elif escolha == '3':
            # Adiciona os arquivos na pasta local do Colab.
            adicionar_novos_arquivos(PASTA_ORIGEM_LOCAL)
            # Define a pasta local como a origem padrão para a próxima organização.
            pasta_origem_atual = PASTA_ORIGEM_LOCAL
            print(f"Lembrete: A pasta de origem foi definida para '{PASTA_ORIGEM_LOCAL}'. Use a opção 2 para organizar.")

        # --- Opção 4: Renomear ---
        elif escolha == '4':
            print("\n--- Renomear Arquivo ---")
            nome_antigo = input("Digite o nome EXATO do arquivo que deseja renomear (com extensão, ex: tese_2021.pdf): ")
            nome_novo = input(f"Digite o NOVO nome para o arquivo '{nome_antigo}' (com extensão): ")
            # Verifica se o usuário realmente digitou os dois nomes.
            if nome_antigo != '' and nome_novo != '':
                buscar_e_renomear_arquivo(PASTA_DESTINO, nome_antigo, nome_novo)
            else:
                print("ERRO: Os nomes não podem ser vazios.")

        # --- Opção 5: Remover ---
        elif escolha == '5':
            print("\n--- Remover Arquivo ---")
            nome_para_remover = input("Digite o nome EXATO do arquivo que deseja remover (com extensão): ")
            # Verifica se o usuário digitou um nome.
            if nome_para_remover != '':
                buscar_e_remover_arquivo(PASTA_DESTINO, nome_para_remover)
            else:
                print("ERRO: O nome do arquivo não pode ser vazio.")

        # --- Opção 6: Sair ---
        elif escolha == '6':
            print("\nObrigado por usar o sistema. Até logo!")
            break # O comando 'break' interrompe o laço 'while', encerrando o programa.

        # --- Opção Inválida ---
        else:
            print("\nERRO: Opção inválida. Por favor, escolha um número de 1 a 6.")

In [ ]:
# --- EXECUTA O PROGRAMA ---
# Chama a função do menu principal para começar a interação com o usuário.
menu_principal()